# 特征列

特征列是原始数据和estimator之间的媒介

![](img/0030-1.png)

要定义特征列，需要tf.feature_column模块，这个模块有9个函数

![](img/0030-2.png)

In [2]:
import tensorflow as tf

## numeric_column

In [3]:
# Represent a 10x5 matrix in which each cell contains a tf.float32.
matrix_feature_column = tf.feature_column.numeric_column(key="MyMatrix",
                                                         shape=[10,5], dtype = tf.float64)

numeric_column默认使用float32作为数据类型，可以使用dtype指定。特征默认是标量，可以使用shape指定。

## 分桶列

对数值列分桶可以将单个特征分为多个，将会学习多个权重，会增加模型非线性

In [5]:
# First, convert the raw input to a numeric column.
numeric_feature_column = tf.feature_column.numeric_column("Year")

# Then, bucketize the numeric column on the years 1960, 1980, and 2000.
bucketized_feature_column = tf.feature_column.bucketized_column(
    source_column = numeric_feature_column,
    boundaries = [1960, 1980, 2000])

![](img/0030-3.png)

## 分类标识列

分类标识列类似于分桶列，但是分类标识列中的每个桶标识一个整数

In [7]:
# Create categorical output for an integer feature named "my_feature_b",
# The values of my_feature_b must be >= 0 and < num_buckets
identity_feature_column = tf.feature_column.categorical_column_with_identity(
    key='my_feature_b',
    num_buckets=4) # Values [0, 4)

使用以上分类标识列，input_fn必须包含my_feature_b为key，其值必须属于{0,1,2,3}

## 分类词汇列

分类词汇列会将单词映射成onehot形式

![](img/0030-4.png)

In [10]:
vocabulary_feature_column = tf.feature_column.categorical_column_with_vocabulary_list(
        key='feature_a',
        vocabulary_list=["kitchenware", "electronics", "sports"])

In [11]:
vocabulary_feature_column = tf.feature_column.categorical_column_with_vocabulary_file(
        key='feature_a',
        vocabulary_file="product_class.txt",
        vocabulary_size=3)

可以指定词典，也可以指定词典文件

## 经过哈希处理的列

当类别特别多的时候，比如分类词特别多的时候，使用前面的方法会特别消耗内存，而且有时候也不能穷举所有的类别，这个时候可以将分类词分桶

`tf.feature_column.categorical_column_with_hash_bucket`会首先将分类词hash到一个整数，然后将这个整数对桶数取余得到桶号

In [14]:
hashed_feature_column = tf.feature_column.categorical_column_with_hash_bucket(
        key = "some_feature",
        hash_bucket_size = 100) # The number of categories

这种处理方式在大规模数据上训练模型非常有用，如广告模型。当多个特征hash到同一个桶中的时候，可以强迫模型去学习其他特征来区分样本，增加模型泛化性。

## 组合列

机器学习模型里面，有实际意义的组合特征比单个特征更有用，在广告模型中特别常见，`tf.feature_column.crossed_column `支持组合特征

In [ ]:
def make_dataset(latitude, longitude, labels):
    assert latitude.shape == longitude.shape == labels.shape
    features = {'latitude': latitude.flatten(),
                'longitude': longitude.flatten()}
    labels=labels.flatten()
    return tf.data.Dataset.from_tensor_slices((features, labels))

# Bucketize the latitude and longitude using the `edges`
latitude_bucket_fc = tf.feature_column.bucketized_column(
    tf.feature_column.numeric_column('latitude'),
    list(atlanta.latitude.edges))

longitude_bucket_fc = tf.feature_column.bucketized_column(
    tf.feature_column.numeric_column('longitude'),
    list(atlanta.longitude.edges))

# Cross the bucketized columns, using 5000 hash bins.
crossed_lat_lon_fc = tf.feature_column.crossed_column(
    [latitude_bucket_fc, longitude_bucket_fc], 5000)

hash过程大概是：
feature_id = Hash64(second_value, Hash64(first_value)) % hash_bucket_size

详细介绍参见：https://www.tensorflow.org/api_docs/python/tf/feature_column/crossed_column

特征组合后将会以下面的特征进行分类：

In [ ]:
fc = [
    latitude_bucket_fc,
    longitude_bucket_fc,
    crossed_lat_lon_fc]

# Build and train the Estimator.
est = tf.estimator.LinearRegressor(fc, ...)

有些反直觉的是，在创建特征组合时，通常仍应在模型中包含原始（未组合）特征（如前面的代码段中所示）。独立的纬度和经度特征有助于模型区分组合特征中发生哈希冲突的样本。